Revisions:

* 07/16/2016 -- Updated for ncclient 0.5.2

# GSP SEVT NETCONF/YANG

This notebook goes through a set of examples with a XRv9000 virtual platform. The goal is to show how NETCONF/YANG can be simply leveraged with IOS-XR to perform a range of tasks for the customer. We will cover topics like:

* Basic connectivity
* Why we really want to use some form of client library
* Getting started with a Python client like [ncclient](https://github/com/ncclient/ncclient)
* Model and feature discovery
* Querying and configuring features

First, let's talk a little about the environment being used here. This is all run inside **jupyter notebooks**, which some may already be familiar with as **iPython notebooks**.

A jupyter notebook lets us mix text and code and provides a simple way to experiment with Python code.

## Useful Snippets

Let's define some useful little snippets of code that we might want to run to define utilities or to enable debugs, set variables for use, etc.

### Enable Debugging

The ncclient libary can generate vast amounts of debug information via standard Python logging. The code fragment below shows how to enable this. But the logging is **vast**! You have been warned!

In [ ]:
import logging

handler = logging.StreamHandler()
for l in ['ncclient.transport.ssh', 'ncclient.transport.session', 'ncclient.operations.rpc']:

    logger = logging.getLogger(l)
    logger.addHandler(handler)
    logger.setLevel(logging.DEBUG)

## Connecting to a Device

First, we need to make sure the device itself has NETCONF/YANG enabled. It's pretty simple:

```
ssh server v2
ssh server netconf
netconf-yang agent ssh
```

You also need to generate a host key from the exec prompt:

```
crypto key generate rsa
```

Next, let's define some variables that let us attach to the device. I'm connecting through an SSH tunnel to my device running on a UCS server in the lab. You should replace the variables below with the device of your choice:

In [ ]:
# Running over an SSH tunnel!
HOST = '127.0.0.1'
PORT = 8303
USER = 'cisco'
PASS = 'cisco'

In [ ]:
# Mark's ASR1K
HOST = '172.27.255.40'
PORT = 830
USER = 'admin'
PASS = 'admin'

In [ ]:
# This is the router you will use for this example notebook, and maybe others.
HOST = '172.26.170.253'
PORT = 830
USER = 'admin'
PASS = 'C1sdevops'

In [ ]:
# running locally in a vagrant box
HOST = '127.0.0.1'
PORT = 2223
USER = 'vagrant'
PASS = 'vagrant'

In [1]:
# Benoit's router!
HOST = '172.27.74.13'
PORT = 830
USER = 'lab'
PASS = 'lab'

Now let's establish a NETCONF session to that box using ncclient. Set the timeout to 600 seconds to allow for long-running operations.

In [2]:
from ncclient import manager
from lxml import etree

def pretty_print(nc_retval):
    '''The ncclient library works with XML. It takes XML in and gives XML back.
    Yeah, I know it's not JSON, so we need to make it a little easier to read,
    so let's define a pretty printer we can use with ncclient responses
    '''
    print(etree.tostring(nc_retval.data_ele, pretty_print=True))

def my_unknown_host_cb(host, fingerprint):
    return True

m = manager.connect(host=HOST, port=PORT, username=USER, password=PASS,
                    timeout=600,
                    allow_agent=False,
                    look_for_keys=False,
                    hostkey_verify=False,
                    unknown_host_cb=my_unknown_host_cb)

## Capabilities

Let's look at the capabilities presented by the thing we've just connected to:

In [ ]:
for item in iter(m.server_capabilities):
    print(item)

Ok, that's a bit messy, so let's tidy it up a bit and look, initially, at all the base netconf capabilities:

In [ ]:
nc_caps = [c for c in m.server_capabilities if c.startswith('urn:ietf:params:netconf')]
for c in nc_caps:
    print(c)

And now let's look at the capabilities that are related to model support:

In [3]:
import re

for c in m.server_capabilities:
    model = re.search('module=([^&]*)&', c)
    if model is not None:
        print("{}".format(model.group(1)))

        revision = re.search('revision=([0-9]{4}-[0-9]{2}-[0-9]{2})', c)
        if revision is not None:
            print("  revision = {}".format(revision.group(1)))

        deviations = re.search('deviations=([a-zA-Z0-9\-,]+)($|&)',c)
        if deviations is not None:
            print("  deviations = {}".format(deviations.group(1)))

        features = re.search('features=([a-zA-Z0-9\-,]+)($|&)',c)
        if features is not None:
            print("  features = {}".format(features.group(1)))

Cisco-IOS-XR-infra-correlator-oper
  revision = 2015-11-09
openconfig-vlan
  revision = 2015-10-09
Cisco-IOS-XR-infra-sla-datatypes
  revision = 2015-11-09
Cisco-IOS-XR-ppp-ma-ssrp-cfg
  revision = 2015-11-09
Cisco-IOS-XR-shellutil-filesystem-oper
  revision = 2015-11-09
cisco-xr-openconfig-interfaces-deviations
  revision = 2016-05-16
Cisco-IOS-XR-ip-iarm-v6-oper
  revision = 2015-11-09
Cisco-IOS-XR-plat-chas-invmgr-oper
  revision = 2015-01-07
Cisco-IOS-XR-ipv6-acl-cfg
  revision = 2015-11-09
Cisco-IOS-XR-infra-systemmib-cfg
  revision = 2015-11-09
Cisco-IOS-XR-infra-correlator-cfg
  revision = 2015-11-09
Cisco-IOS-XR-ipv4-acl-datatypes
  revision = 2015-11-09
Cisco-IOS-XR-asr9k-lc-ethctrl-cfg
  revision = 2015-11-09
Cisco-IOS-XR-ipv6-ma-oper
  revision = 2015-10-20
Cisco-IOS-XR-tty-management-cfg
  revision = 2015-09-25
Cisco-IOS-XR-infra-tc-oper
  revision = 2015-11-09
Cisco-IOS-XR-pbr-vservice-ea-oper
  revision = 2015-11-09
Cisco-IOS-XR-man-netconf-cfg
  revision = 2016-03-15
Cis

## Schema Discovery

Building on the parsing of the capabilities we saw above, NETCONF/YANG can also let a client discover more details on the schemas supported by a box.

But why do we care? Let's think back to what we talked about earlier. About the need for boxes to describe themselves to their clients. To expose their "model".

Let's pick a base model that looks like it may do something interesting, for example ```Cisco-IOS-XR-ifmgr-cfg```, and let's download the schema. The ncclient library provides a nice, simple function for that.

In [9]:
SCHEMA_TO_GET = 'openconfig-bgp'
c = m.get_schema(SCHEMA_TO_GET)
print c.data

module openconfig-bgp {

  yang-version "1";

  // namespace
  namespace "http://openconfig.net/yang/bgp";

  prefix "bgp";

  // import some basic inet types
  import ietf-inet-types { prefix inet; }
  import openconfig-bgp-multiprotocol { prefix bgp-mp; }
  import openconfig-routing-policy { prefix rpol; }
  import openconfig-bgp-types { prefix bgp-types; }
  import openconfig-bgp-operational { prefix bgp-op; }
  import openconfig-extensions { prefix oc-ext; }

  // meta
  organization
    "OpenConfig working group";

  contact
    "OpenConfig working group
    netopenconfig@googlegroups.com";

  description
    "This module describes a YANG model for BGP protocol
    configuration.It is a limited subset of all of the configuration
    parameters available in the variety of vendor implementations,
    hence it is expected that it would be augmented with vendor-
    specific configuration data as needed. Additional modules or
    submodules to handle other aspects of BGP configuration

Maybe we can use pyang to do something more interesting?

In [11]:
from subprocess import Popen, PIPE, STDOUT

SCHEMA_TO_GET = 'openconfig-bgp'
c = m.get_schema(SCHEMA_TO_GET)
p = Popen(['pyang', '-f', 'tree'], stdout=PIPE, stdin=PIPE, stderr=PIPE)
stdout_data = p.communicate(input=c.data)[0]

print stdout_data

module: openconfig-bgp
   +--rw bgp!
      +--rw global
      |  +--rw config
      |  |  +--rw as           inet:as-number
      |  |  +--rw router-id?   inet:ipv4-address
      |  +--ro state
      |  |  +--ro as           inet:as-number
      |  |  +--ro router-id?   inet:ipv4-address
      |  +--rw default-route-distance
      |  |  +--rw config
      |  |  |  +--rw external-route-distance?   uint8
      |  |  |  +--rw internal-route-distance?   uint8
      |  |  +--ro state
      |  |     +--ro external-route-distance?   uint8
      |  |     +--ro internal-route-distance?   uint8
      |  +--rw confederation
      |  |  +--rw config
      |  |  |  +--rw enabled?      boolean
      |  |  |  +--rw identifier?   inet:as-number
      |  |  |  +--rw member-as*    inet:as-number
      |  |  +--ro state
      |  |     +--ro enabled?      boolean
      |  |     +--ro identifier?   inet:as-number
      |  |     +--ro member-as*    inet:as-number
      |  +--rw graceful-restart
      |  |  

Or we can use ```ietf-netconf-monitoring```:

In [ ]:
import jxmlease

filter = '''
<netconf-state xmlns="urn:ietf:params:xml:ns:yang:ietf-netconf-monitoring">
  <schemas/>
</netconf-state>'''

c = m.get(filter=('subtree', filter))
r = jxmlease.parse(c.data_xml)
noncisco = set()
for s in r[u'data'][u'netconf-state'][u'schemas'][u'schema']:
    ident = s[u'identifier']
    if 'Cisco' not in ident and 'cisco' not in ident and '-MIB' not in ident:
        noncisco.add(ident)
for s in sorted(noncisco):
    print(s)

## What About Config? (On IOS-XR)

The ncclient library provides for some simple operations. Let's skip thinking about schemas and stuff like that. Instead let's focus on config and getting end setting it. For that, ncclient provides two methods:

* get_config - takes a target data store and an optional filter
* edit_config - takes a target data store and an XML document with the edit request


### Getting Config

Let's look at some simple requests...

In [ ]:
c = m.get_config(source='running')
pretty_print(c)

#### Minor Digression

Let's try parsing release with ```jxmlease```, and see if it's as easy as JSON. Be sure to have done a ```pip install jxmlease``` in the environment you are running Jupyter in!

In [ ]:
import jxmlease

c = m.get_config(source='running')
r = jxmlease.parse(c.data_xml)

for intf in r[u'data'][u'interface-configurations'][u'interface-configuration']:
    if 'ipv4-network' in intf.keys():
        print("{} has IPv4 address {}".format(
                intf[u'interface-name'],
                intf[u'ipv4-network'][u'addresses'][u'primary'][u'address']))
    else:
        print("{} has no IPv4 network config".format(intf[u'interface-name']))


#### Back to getting config...

Now let's add in a simple filter to just get interface configuration:

In [ ]:
filter = '''
<interface-configurations xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ifmgr-cfg"/>
'''
c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

Let's refine the query a bit to look at **just** the management ethernet port:

In [ ]:
filter = '''
<interface-configurations xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ifmgr-cfg">
  <interface-configuration>
    <active>act</active>
    <interface-name>GigabitEthernet0/0/0/0</interface-name>
  </interface-configuration>
</interface-configurations>
'''

c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

Or we may want to look at just ISIS config:

In [ ]:
filter = '''<isis xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-clns-isis-cfg"/>'''
c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

Or maybe just policy configuration?

In [ ]:
filter = '''<policy-manager xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-infra-policymgr-cfg"/>'''
c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

IP-SLA config anyone?

In [ ]:
filter = '''<sla xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-infra-sla-cfg"/>'''
c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

### Editing Config

Let's start with something simple, like applying an IP address to an interface and bringing it up. We can actually use the data we got from the management ethernet port as a template.

To do this, we use two functions:

* ```edit_config``` on the candidate data store
* ```commit``` to commit the change to running config, just like on the CLI

For good measure, we'll also get the interface config back again to check out that what we asked to happen, actually did happen.

In [ ]:
from ncclient.operations import TimeoutExpiredError

edit_data = '''
<config>
  <interface-configurations xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ifmgr-cfg">
   <interface-configuration>
    <active>act</active>
    <interface-name>GigabitEthernet0/0/0/0</interface-name>
    <ipv4-network xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ipv4-io-cfg">
     <addresses>
      <primary>
       <address>192.168.1.10</address>
       <netmask>255.255.255.0</netmask>
      </primary>
     </addresses>
    </ipv4-network>
    <shutdown nc:operation="remove"/>
   </interface-configuration>
  </interface-configurations>
</config>
'''

filter = '''
<interface-configurations xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ifmgr-cfg">
  <interface-configuration>
    <interface-name>GigabitEthernet0/0/0/0</interface-name>
  </interface-configuration>
</interface-configurations>
'''

try:
    m.edit_config(edit_data, target='candidate', format='xml')
    m.commit()
    c = m.get_config(source='running', filter=('subtree', filter))
    pretty_print(c)
except TimeoutExpiredError as e:
    print("Operation timeout!")
except Exception as e:
    print("severity={}, tag={}".format(e.severity, e.tag))
    print(e)

And now what about deleting that config and taking the interface back to it's original state?

In [ ]:
from ncclient.operations import TimeoutExpiredError

filter = '''
<interface-configurations xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ifmgr-cfg">
  <interface-configuration>
    <interface-name>GigabitEthernet0/0/0/0</interface-name>
  </interface-configuration>
</interface-configurations>
'''

edit_data1 = '''
<config>
  <interface-configurations xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ifmgr-cfg">
   <interface-configuration>
    <active>act</active>
    <interface-name>GigabitEthernet0/0/0/0</interface-name>
    <ipv4-network xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ipv4-io-cfg" nc:operation="remove"/>
    <shutdown/>
   </interface-configuration>
  </interface-configurations>
</config>
'''


edit_data2 = '''
<config>
  <interface-configurations xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ifmgr-cfg">
   <interface-configuration>
    <active>act</active>
    <interface-name>GigabitEthernet0/0/0/0</interface-name>
    <shutdown nc:operation="delete"/>
    <ipv4-network xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ipv4-io-cfg" nc:operation="delete"/>
   </interface-configuration>
  </interface-configurations>
</config>
'''


try:
    m.edit_config(edit_data1, target='candidate', format='xml')
    m.commit()
    c = m.get_config(source='running', filter=('subtree', filter))
    pretty_print(c)
except TimeoutExpiredError as e:
    print("Operation timeout!")
except Exception as e:
    print("severity={}, tag={}".format(e.severity, e.tag))
    print(e)

### Create An Interface

Let's also deal with some pre-config issues here as well!!

In [ ]:
from ncclient.operations import TimeoutExpiredError

filter_1000 = '''
<interface-configurations xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ifmgr-cfg">
  <interface-configuration>
    <interface-name>GigabitEthernet1/0/0/0</interface-name>
  </interface-configuration>
</interface-configurations>
'''


filter_intf = '''
<interface-configurations xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ifmgr-cfg">
</interface-configurations>
'''


edit_data_cr_1000 = '''
<config>
  <interface-configurations xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ifmgr-cfg">
   <interface-configuration>
    <active>act</active>
    <interface-name>GigabitEthernet1/0/0/0</interface-name>
    <ipv4-network xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ipv4-io-cfg">
     <addresses>
      <primary>
       <address>192.168.3.10</address>
       <netmask>255.255.255.0</netmask>
      </primary>
     </addresses>
    </ipv4-network>
   </interface-configuration>
  </interface-configurations>
</config>
'''


edit_data_del_1000 = '''
<config>
  <interface-configurations xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ifmgr-cfg">
   <interface-configuration nc:operation="delete">
    <active>pre</active>
    <interface-name>GigabitEthernet1/0/0/0</interface-name>
   </interface-configuration>
  </interface-configurations>
</config>
'''


try:
    m.edit_config(edit_data_del_1000, target='candidate', format='xml')
    m.commit()
    c = m.get_config(source='running', filter=('subtree', filter_intf))
    pretty_print(c)
except TimeoutExpiredError as e:
    print("Operation timeout!")
except Exception as e:
    print("severity={}, tag={}".format(e.severity, e.tag))
    print(e)

## What About Oper Data?

As we touched on before, NETCONF also has the ```get``` operation. This can get both configuration state **and** operational state.

### Verbose Interface Stats

In [ ]:
from ncclient.operations import TimeoutExpiredError

filter = '''
<interfaces xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-pfi-im-cmd-oper"/>
'''

try:
    c = m.get(filter=('subtree', filter))
    pretty_print(c)
except TimeoutExpiredError as e:
    print("Operation timeout!")
except Exception as e:
    print("severity={}, tag={}".format(e.severity, e.tag))
    print(e)

### Slightly Less Verbose Interface Stats

In [ ]:
from ncclient.operations import TimeoutExpiredError

filter = '''
<interfaces xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-pfi-im-cmd-oper">
  <interface-briefs/>
</interfaces>
'''

try:
    c = m.get(filter=('subtree', filter))
    pretty_print(c)
except TimeoutExpiredError as e:
    print("Operation timeout!")
except Exception as e:
    print("severity={}, tag={}".format(e.severity, e.tag))
    print(e)

### Zeroing In On A Specific Interface

In [ ]:
from ncclient.operations import TimeoutExpiredError

filter = '''
<interfaces xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-pfi-im-cmd-oper">
  <interface-xr>
    <interface>
      <interface-name>MgmtEth0/RP0/CPU0/0</interface-name>
      <interface-statistics/>
    </interface>
  </interface-xr>
</interfaces>
'''

try:
    c = m.get(filter=('subtree', filter))
    pretty_print(c)
except TimeoutExpiredError as e:
    print("Operation timeout!")
except Exception as e:
    print("severity={}, tag={}".format(e.severity, e.tag))
    print(e)

### Zeroing In On Specific Stats Across Interfaces

In [ ]:
from ncclient.operations import TimeoutExpiredError

filter = '''
<interfaces xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-pfi-im-cmd-oper">
  <interface-xr>
    <interface>
      <interface-name>MgmtEth0/RP0/CPU0/0</interface-name>
      <interface-statistics>
        <full-interface-stats>
          <packets-received/>
          <bytes-received/>
          <packets-sent/>
          <bytes-sent/>
        </full-interface-stats>
      </interface-statistics>
    </interface>
  </interface-xr>
</interfaces>
'''

try:
    c = m.get(filter=('subtree', filter))
    pretty_print(c)
except TimeoutExpiredError as e:
    print("Operation timeout!")
except Exception as e:
    print("severity={}, tag={}".format(e.severity, e.tag))
    print(e)

## Conclusion

NETCONF presents a number of useful primitives that we looked at today:

* get config
* get
* edit config
* commit
* get schema

There are more, so follow up to understand more later.

With the prmitives we ran through today, you can do basic model discovery, understand what features are supported, and understand what parts of models are not supported.

Products such as NSO, or open source projects like ODL or the YDK, can use these basic capabilities to work with devices in a much more reliable way.

## IOS-XE Samples

The following samples will only work with IOS-XE platforms.

### Get-Config and Get

In [ ]:
filter = '''/native/vrf/definition[name="Mgmt-intf"]'''
c = m.get_config(source='running', filter=('xpath', filter))
pretty_print(c)

In [ ]:
filter = '''/netconf-state/schemas'''
c = m.get(filter=('xpath', filter))
pretty_print(c)

In [ ]:
filter = '''
<native xmlns="http://cisco.com/ns/yang/ned/ios">
 <vrf>
  <definition>
   <name>Mgmt-intf</name>
  </definition>
 </vrf>
</native>'''
c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

In [ ]:
filter = '''/routing-state/routing-instance/routing-protocols/routing-protocol/ospf'''
c = m.get(filter=('xpath', filter))
pretty_print(c)

In [ ]:
filter = '''
<routing-state xmlns="urn:ietf:params:xml:ns:yang:ietf-routing">
 <routing-instance>
  <routing-protocols>
   <routing-protocol>
    <ospf xmlns="urn:ietf:params:xml:ns:yang:ietf-ospf"/>
   </routing-protocol>
  </routing-protocols>
 </routing-instance>
</routing-state>'''
c = m.get(filter=('subtree', filter))
pretty_print(c)

### Edit-Config

In [ ]:
from ncclient.operations import TimeoutExpiredError

filter = '''
 <native xmlns="http://cisco.com/ns/yang/ned/ios">
  <interface>
   <Loopback/>
  </interface>
 </native>
'''

edit_data = '''<config>
 <native xmlns="http://cisco.com/ns/yang/ned/ios">
  <interface>
   <Loopback nc:operation="delete">
    <name>200</name>
    <description>temp</description>
   </Loopback>
 </native>
</config>'''

try:
    m.edit_config(edit_data, target='running', format='xml')
    c = m.get_config(source='running', filter=('subtree', filter))
    pretty_print(c)
except TimeoutExpiredError as e:
    print("Operation timeout!")
except Exception as e:
    # print("severity={}, tag={}".format(e.severity, e.tag))
    print(e)

### More Edit Config

In [ ]:
filter = '''
 <native xmlns="http://cisco.com/ns/yang/ned/ios">
  <interface>
   <Loopback>
    <name>200</name>
   </Loopback>
  </interface>
 </native>
'''

edit_data = """
                <config>
                <native xmlns="http://cisco.com/ns/yang/ned/ios">
                 <interface>
                  <Loopback>
                   <name>200</name>
                   <ip>
                    <address>
                        <primary>
                            <address>55.55.55.55</address>
                            <mask>255.255.255.0</mask>
                        </primary>
                        <secondary>
                            <address>9.9.9.9</address>
                            <mask>255.255.255.0</mask>
                            <secondary/>
                        </secondary>
                        <secondary>
                            <address>68.1.1.1</address>
                            <mask>255.255.255.0</mask>
                            <secondary/>
                        </secondary>
                    </address>
                   </ip>
                  </Loopback>
                 </interface>
                </native>
                </config>
"""

try:
    m.edit_config(edit_data, target='running', format='xml')
    c = m.get_config(source='running', filter=('subtree', filter))
    pretty_print(c)
except TimeoutExpiredError as e:
    print("Operation timeout!")
except Exception as e:
    # print("severity={}, tag={}".format(e.severity, e.tag))
    print(e)